In [20]:
from psql_helpers import get_session, read_table_async
from psql_models import Thread, User

In [24]:
from pytz import timezone

In [27]:
timezone('utc')

<UTC>

In [21]:
r = await read_table_async(User)

In [23]:
r

,id,email,username,first_name,last_name,image,created_timestamp,updated_timestamp
0,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,2280905@gmail.com,string,string,string,None,2024-11-06 09:48:34.231147+00:00,2024-11-06 09:48:34.231150+00:00
1,aa8d9413-8d77-437f-96f1-0187bc9d412f,ryabenko.sergey@gmail.com,string,string,string,None,2024-11-06 12:52:16.721608+00:00,2024-11-06 12:52:16.721611+00:00
2,f05bd139-3fa1-4563-a900-2516dffb67b3,dmitrygrankin0@gmail.com,string,string,string,None,2024-11-07 09:01:39.836940+00:00,2024-11-07 09:01:39.836943+00:00
3,92c90e8d-6ab9-416c-a906-0e5c58e82153,v.tsurenkova@citydrive.ru,string,string,string,None,2024-11-07 09:07:39.483721+00:00,2024-11-07 09:07:39.483724+00:00
4,a97b05c3-ed1b-40f3-bf71-ba6937b4da39,iaprobotsamuel@gmail.com,string,string,string,None,2024-11-07 09:09:35.279943+00:00,2024-11-07 09:09:35.279947+00:00
5,fcd46b26-ddd4-4bd2-99d4-24629fea46a7,sota197508@gmail.com,None,None,None,None,2024-11-07 09:11:21.020094+00:00,2024-11-07 09:11:21.020096+00:00
6,058980af-dfa0-4dc1-b9c0-adf6c24e4994,evgenia.grudtseva@oom-agency.ru,Евгения Грудцева,Евгения,Грудцева,https://lh3.googleusercontent.com/a/ACg8ocIO7b...,2024-11-07 09:21:14.641562+00:00,2024-11-07 09:21:14.641564+00:00


In [22]:
r['image'].iloc[-1]

'https://lh3.googleusercontent.com/a/ACg8ocIO7bXqB1NMoidwvpbMTTyBYZiQWPk33qXjJuI9k1b3m9pxK_8=s96-c'

In [ ]:


async with get_session() as session:
    await session.execute(Thread.__table__.delete())
    await session.commit()

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from uuid import UUID
from sqlalchemy.ext.asyncio import AsyncSession
from chat import MeetingChatManager
from psql_helpers import get_session

In [3]:
from vexa import VexaAPI
vexa = VexaAPI()
await vexa.get_user_info()
user_id = vexa.user_id


Vexa token: 3ae04e20124d40babc5107e658c666b6
User information retrieved successfully.


In [4]:
from uuid import UUID

In [5]:


# Single meeting chat
meeting_id = UUID('aa87b543-f5c9-426c-a6ed-77b26f93e0da')
async with get_session() as session:
    chat_manager = MeetingChatManager(session)
    await chat_manager.initialize()
    
    # Chat about specific meeting - will create meeting-specific thread
    async for result in chat_manager.chat(
        user_id=user_id,
        meeting_ids=[meeting_id],
        query="50 char on the meeting"
    ):
        if "chunk" in result:
            print(result["chunk"], end="")
        elif "thread_id" in result:
            thread_id = result["thread_id"]  # Store for continued conversation

16:05:46 - vexa_api - INFO - Initializing ChatManager


Dmitriy completed API; frontend integration ongoing.

In [6]:
thread_id

'cf75f12e-5205-434a-abad-6e9540ededf1'

In [7]:
async for result in chat_manager.chat(
    user_id=user_id,
    meeting_ids=[meeting_id],
    thread_id=thread_id,
    query="What were the key decisions?"
):
    print(result.get("chunk", ""), end="")

- **API Development Completion**: Dmitriy Grankin confirmed the completion of the API, which is now ready for frontend integration.

- **Frontend Integration**: Ongoing work on integrating the frontend with the API, with a focus on ensuring elements are clickable and functional.

- **Model Testing**: Decision to test the LAMA 3.2 model with 90 billion parameters, which has shown promising results but requires further refinement.

- **Authorization Method**: Agreement to implement Google authorization for user access, utilizing LocalStorage for storing user tokens.

- **Error Handling**: Acknowledgment of issues related to 'incomplete chunk encoding' errors, necessitating further investigation and resolution.

- **Data Indexing**: Plan to index data on the dashboard to facilitate the display of information related to calls and chats, enhancing user experience.

- **Design Considerations**: Discussion on the importance of adhering to a design system for the frontend, ensuring consistency

In [8]:
async for result in chat_manager.chat(
    user_id=user_id,
    meeting_ids=[meeting_id],
    thread_id=thread_id,
    query="in a few words"
):
    print(result.get("chunk", ""), end="")

Dmitriy completed API; frontend integration ongoing; testing LAMA 3.2 model; addressing authorization and error handling issues.

In [ ]:
async for result in chat_manager.chat(
    user_id=user_id,
    meeting_ids=[meeting_id],
    thread_id=thread_id,
    query=""
):
    print(result.get("chunk", ""), end="")

In [13]:
from chat import SearchChatManager

In [14]:

async with get_session() as session:
    chat_manager = SearchChatManager(session)
    await chat_manager.initialize()

TypeError: ChatManager.__init__() takes 1 positional argument but 2 were given

In [ ]:
# Follow up in global chat
async for result in chat_manager.chat(
    user_id=user_id,
    meeting_ids=None,
    thread_id=None,
    query="Can you help me analyze meetings?"
):
    print(result.get("chunk", ""), end="")

In [ ]:

# Chat about multiple meetings
async for result in chat_manager.chat(
    user_id=user_id,
    meeting_ids=[UUID('id1'), UUID('id2')],
    query="Compare these meetings"
):
    print(result.get("chunk", ""), end="")